In [ ]:
import re

def nonestr(string):
    if string is None:
        return ""
    return string

class StringModel:
    def __init__(self, model, default=dict(), global_default="", sep="_.", escape="$"):
        """
        The escape parameter shall never be used in the values
        This class is not made for proper handling of escape mechanism
        """
        assert len(escape) == 1
        self.model = model
        self.default = default
        self.global_default = global_default
        self.sep = sep
        self.escape = escape
        
    def _split(self, string):
        pat = f"(?:^|([^{self.escape}]))[{self.sep}]"
        res = re.split(pat, string)
        size = int((len(res) + 1) / 2)
        res += ['']
        return [res[i * 2] + nonestr(res[i * 2 + 1]) for i in range(size)]
    
    def _parse_substring(self, substring):
        pat ='([\w\d]+)(?:{([\w\d]+)})?'
        return re.findall(pat, substring)[0]
    
    def _get_slots(self):
        substrings = self._split(self.model)
        return [self._parse_substring(substring) for substring in substrings]
    
    def fill(self, **params):
        values = {slotname: self.global_default for _, slotname in self._get_slots()}
        values.update(self.default)
        values.update(params)
        return self.model.format(**values)
    
    def _extract_slot(self, string, prefix):
        pat = f"(?:^|[{self.sep}])({prefix}[^{self.sep}]*)(?:$|[{self.sep}])"
        return re.findall(pat, string)[0]
    
    def extract(self, string):
        slots = filter(lambda x: x[1] != "", self._get_slots())
        substrings = self._split(string)
        dict_res = dict()
        for prefix, slotname in slots:
            for substring in substrings:
                if re.match(f"{prefix}.*", substring):
                    dict_res[slotname] = re.findall(f"{prefix}(.*)", substring)[0]
                    break
        
        return dict_res

In [15]:
sm = StringModel("vac{ho}_messy{ok}.txt")

In [16]:
sm.fill(ho="salut")

'vacsalut_messy.txt'

In [17]:
sm.extract('vacsalut_messy.txt')

{'ho': 'salut', 'ok': ''}